In [3]:
import os
import shutil
import random

def create_subset_split(base_path, target_path, images_per_class=100):
    classes = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    
    train_n = int(images_per_class * 0.7)
    val_n = int(images_per_class * 0.15)
    
    for split in ['train', 'val', 'test']:
        for cls in classes:
            os.makedirs(os.path.join(target_path, split, cls), exist_ok=True)

    for cls in classes:
        cls_path = os.path.join(base_path, cls)
        all_images = os.listdir(cls_path)
        
        random.seed(42)
        random.shuffle(all_images)
        subset = all_images[:images_per_class]

        train_imgs = subset[:train_n]
        val_imgs = subset[train_n:train_n + val_n]
        test_imgs = subset[train_n + val_n:]

        for split_name, img_list in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            for img in img_list:
                shutil.copy(
                    os.path.join(cls_path, img),
                    os.path.join(target_path, split_name, cls, img)
                )
                
    print(f"Created subset: {images_per_class} images per class.")
    print(f"Structure: 70 Train | 15 Val | 15 Test per category.")

create_subset_split('data', 'satellite_subset', images_per_class=100)

Created subset: 100 images per class.
Structure: 70 Train | 15 Val | 15 Test per category.


In [5]:
import numpy as np

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        raise NotImplementedError

    def backward(self, output_gradient, learning_rate):
        raise NotImplementedError

In [6]:
class ConvLayer(Layer):
    def __init__(self, input_shape, kernel_size, depth):
      
        self.input_shape = input_shape
        self.input_depth, self.input_height, self.input_width = input_shape
        self.depth = depth
        self.kernel_size = kernel_size
        
        self.output_shape = (depth, self.input_height - kernel_size + 1, self.input_width - kernel_size + 1)
        
        self.kernels = np.random.randn(depth, self.input_depth, kernel_size, kernel_size) * np.sqrt(2/self.input_depth)
        self.biases = np.zeros(self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        
        for d in range(self.depth):
            for i in range(self.input_depth): 
                self.output[d] += self._correlate(self.input[i], self.kernels[d, i])
        return self.output

    def _correlate(self, img, kernel):
        h, w = img.shape
        k = self.kernel_size
        out_h, out_w = h - k + 1, w - k + 1
        res = np.zeros((out_h, out_w))
        
        for y in range(out_h):
            for x in range(out_w):
                res[y, x] = np.sum(img[y:y+k, x:x+k] * kernel)
        return res

In [7]:
class ActivationLayer(Layer):
    def forward(self, input):
        self.input = input
        return np.maximum(0, input)

    def backward(self, output_gradient, learning_rate):
        return output_gradient * (self.input > 0)

In [8]:
class MaxPool(Layer):
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, input):
        self.input = input
        self.depth, self.input_height, self.input_width = input.shape
        self.output_height = (self.input_height - self.kernel_size) // self.stride + 1
        self.output_width = (self.input_width - self.kernel_size) // self.stride + 1
        
        output = np.zeros((self.depth, self.output_height, self.output_width))
        
        for d in range(self.depth):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    h_start = i * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = j * self.stride
                    w_end = w_start + self.kernel_size
                    
                    output[d, i, j] = np.max(input[d, h_start:h_end, w_start:w_end])
        return output

    def backward(self, output_gradient, learning_rate):
        d_input = np.zeros(self.input.shape)
        
        for d in range(self.depth):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    h_start = i * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = j * self.stride
                    w_end = w_start + self.kernel_size
                    
                    window = self.input[d, h_start:h_end, w_start:w_end]
                    mask = (window == np.max(window))
                    
                    d_input[d, h_start:h_end, w_start:w_end] += mask * output_gradient[d, i, j]
        return d_input

In [9]:
class Flatten(Layer):
    def forward(self, input):
        self.input_shape = input.shape
        return input.flatten().reshape(1, -1) 

    def backward(self, output_gradient, learning_rate):
        return output_gradient.reshape(self.input_shape)

In [10]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(1/input_size)
        self.biases = np.zeros((1, output_size))

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.biases

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(self.input.T, output_gradient)
        input_gradient = np.dot(output_gradient, self.weights.T)

        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * output_gradient
        
        return input_gradient

In [11]:
class Softmax(Layer):
    def forward(self, input):
        exps = np.exp(input - np.max(input))
        self.output = exps / np.sum(exps)
        return self.output

    def backward(self, output_gradient, learning_rate):
        return output_gradient

In [12]:
def categorical_cross_entropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    return -np.sum(y_true * np.log(y_pred))

def categorical_cross_entropy_gradient(y_true, y_pred):
    return y_pred - y_true

In [13]:
class SatelliteCNN:
    def __init__(self, layers):
        self.layers = layers

    def predict(self, input):
        output = input
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            total_loss = 0
            for x, y in zip(x_train, y_train):
                output = self.predict(x)
                
                total_loss += categorical_cross_entropy(y, output)
                
                gradient = categorical_cross_entropy_gradient(y, output)
                for layer in reversed(self.layers):
                    gradient = layer.backward(gradient, learning_rate)
            
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(x_train):.4f}")

In [14]:
model = SatelliteCNN([
    ConvLayer((3, 32, 32), kernel_size=3, depth=8), 
    ActivationLayer(),
    MaxPool(kernel_size=2, stride=2),               
    Flatten(),                                      
    Dense(1800, 4),                                 
    Softmax()
])